## 1. Daten sammeln

Als erstes werden die Daten von Smartvote abgefragt und abgespeichert. Smartvote stellt dafür eine Schnittstelle (API) zur Verfügung mithilfe dere die Daten abgefragt werden können.
Wir wollen für die letzte Nationalratswahl die Antworten aller Kandidierenden abfragen.

Um mithilfe von Python eine Anfrage an eine Schnittstelle an einem anderen Ort im Internet zu machen, wird die Programmierbibliothek `requests` verwendet.
Diese stellt eine Anfrage zusammen und sendet sie an einen anderen Ort im Internet und kann die entsprechende Antwort verarbeiten.

Die Daten werden im Datenformat [JSON](https://de.wikipedia.org/wiki/JavaScript_Object_Notation) zur Verfügung gestellt.
Ein sehr simples strukturiertes Dateiformat.

Als erstes müssen die verwendeten Libraries importiert werden.
- `requests` für das Anfordern der Daten
- `json` für das Handhaben der gesammelten Daten


In [ ]:
import requests
import json

Als erstes müssen wir ein paar Definitionen machen, welche Informationen wir beschaffen wollen (National- oder Ständeratswahlen) und in welcher Datei wir die Ergebnisse zwischenspeichern wollen.

In [ ]:
mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

electionId = "1057" if mode == 1 else "1084"
filename = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"

Danach kann die Schnittstelle angefragt werden. Wir holen uns über die Schnittstelle so viel Informationen zu jeder kandidierenden Person wie möglich.

In [ ]:
url = "https://backend.smartvote.ch/api/graphql"
headers = {}

body = {
    "operationName": "SearchCandidates",
    "variables": {"searchParams": {"electionId": electionId}},
    "query": """
    query SearchCandidates($searchParams: CandidateSearchParams!) {
        candidates(searchParams: $searchParams) {
            ...SearchCandidate
        }
    }

    fragment SearchCandidate on Candidate {
        id
        firstname
        lastname
        yearOfBirth
        profileImageUrl
        isIncumbent
        isElected
        partyAbbreviation
        partyColor
        hasSmartvoteProfile
        gender

        answers {
            ...Answer
        }
    }

    fragment Answer on Answer {
        questionId
        value
    }
    """,
}


response = requests.post(url, headers=headers, json=body)


if response.status_code == 200:
    candidates = response.json()["data"]["candidates"]
    print(f"Request was successful and returned {len(candidates)} candidate(s)")
else:
    print(f"Request failed with status code {response.status_code}")

Jetzt können wir uns einmal die Informationen zu einer beliebigen Kandidierenden Person darstellen.

In [ ]:
print(json.dumps(candidates[242 if len(candidates) > 242 else 52], indent=4, ensure_ascii=False))

Als nächstes entfernen wir alle Kandidat*innen aus, die kein Smartvote-Profil haben.

In [ ]:
candidates = [candidate for candidate in candidates if candidate["hasSmartvoteProfile"]]

Um die Daten weiterverwenden zu können speichern wir die Resultate in einer eigenen Datei ab.

In [ ]:
# Write the response data to a file
with open(filename, "w", encoding="utf-8") as json_file:
    json.dump(candidates, json_file, indent=4, ensure_ascii=False)

print(f"Response has been written to {filename}.")